In [1]:
import os
import numpy as np
import cv2 
from math import floor

In [2]:
ROOT_DIR = os.path.dirname(os.getcwd())
PROJECT_DIR = os.path.join(ROOT_DIR, 'HaarCascade')
DATA_DIR = os.path.join(PROJECT_DIR, 'dataset')
print(ROOT_DIR)
DEBUG = False

d:\EDS\EDS6397_Digital_Image_Processing\Project


In [3]:
#Get all file name in  dataset/labels
anotations_path = [os.path.join( DATA_DIR, 'labels', x) for x in  os.listdir(os.path.join(DATA_DIR, 'labels'))]
image_paths = [os.path.join( DATA_DIR, 'positive', x.split('.')[0] +'.jpg' ) for x in  os.listdir(os.path.join(DATA_DIR, 'labels'))]
raw_image_paths = [os.path.join( DATA_DIR, 'raw_positive', x.split('.')[0] +'.jpg' ) for x in  os.listdir(os.path.join(DATA_DIR, 'labels'))]
simple_image_paths = [os.path.join( 'positive', x) for x in  os.listdir(os.path.join(DATA_DIR, 'raw_positive'))]


In [4]:
#Get average width and height of all images
widths = []
heights = []
for image_path in raw_image_paths:
    image = cv2.imread(image_path)
    heights.append(image.shape[0])
    widths.append(image.shape[1])

average_width = floor(np.mean(widths))
average_height = floor(np.mean(heights))

print('Average width: ', average_width)
print('Average height: ', average_height)


Average width:  352
Average height:  401


In [5]:
#resize all images to average width and height
for raw_image_path, image_path in zip(raw_image_paths, image_paths):
    image = cv2.imread(raw_image_path)
    image = cv2.resize(image, (average_width, average_height))
    cv2.imwrite(image_path, image)
    

In [6]:
#Get all image paths in dataset/n
n_paths = [os.path.join(  'negative', x) for x in  os.listdir(os.path.join(DATA_DIR, 'negative'))]
print(n_paths)

with open('dataset/bg.txt', 'w') as f:
    for path in n_paths:
        f.write(path + '\n')

['negative\\no0.jpg', 'negative\\no1.jpg', 'negative\\no10.jpg', 'negative\\no100.jpg', 'negative\\no1000.jpg', 'negative\\no1001.jpg', 'negative\\no1002.jpg', 'negative\\no1003.jpg', 'negative\\no1004.jpg', 'negative\\no1005.jpg', 'negative\\no1006.jpg', 'negative\\no1007.jpg', 'negative\\no1008.jpg', 'negative\\no1009.jpg', 'negative\\no101.jpg', 'negative\\no1010.jpg', 'negative\\no1011.jpg', 'negative\\no1012.jpg', 'negative\\no1013.jpg', 'negative\\no1014.jpg', 'negative\\no1015.jpg', 'negative\\no1016.jpg', 'negative\\no1017.jpg', 'negative\\no1018.jpg', 'negative\\no1019.jpg', 'negative\\no102.jpg', 'negative\\no1020.jpg', 'negative\\no1021.jpg', 'negative\\no1022.jpg', 'negative\\no1023.jpg', 'negative\\no1024.jpg', 'negative\\no1025.jpg', 'negative\\no1026.jpg', 'negative\\no1027.jpg', 'negative\\no1028.jpg', 'negative\\no1029.jpg', 'negative\\no103.jpg', 'negative\\no1030.jpg', 'negative\\no1031.jpg', 'negative\\no1032.jpg', 'negative\\no1033.jpg', 'negative\\no1034.jpg', 'ne

In [7]:
def box_check(x, y, w, h, rows, cols):
    if x < 0 or y < 0 or x + w > rows or y + h > cols:
        return False
    return True

In [8]:
with open(os.path.join(DATA_DIR, 'positive.txt'), 'w') as outfile:
    for image_path  in  image_paths:
        
        simple_image_path = 'positive\\' + image_path.split('\\')[-1]
        anotations_path = image_path.replace('positive', 'labels').replace('.jpg', '.txt')

        boxes = []
        with open(anotations_path, 'r') as infile:
            for line in infile:
                x,y,w,h = [float(x) for x in line.split(' ')[1:]]
                #Calculate the x_min,y_min,x_max, y_max in pixel
                x = str(floor((x - w/2) * average_width))
                y = str(floor((y - h/2) * average_height))
                # x = str(int(x * average_width))
                # y = str(int(y * average_height))
                h = str(floor(h * average_height))
                w = str(floor(w * average_width))
            
                # #Draw the rectangle
                # if DEBUG:
                #     print(x_min, y_min, x_max, y_max)
                #     print(x,y,w,h)
                #     cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0,255,0), 2)
                #     cv2.imshow('image', img)
                #     if cv2.waitKey(0) & 0xFF == ord('q'):
                #         cv2.destroyAllWindows()
                #     break
                if box_check(int(x), int(y), int(w), int(h), average_width, average_height):
                    boxes.append(' '.join([x, y, w, h]))
                else: 
                    print(f'Invalid box: { x, y, w, h} at image {simple_image_path}')

        if len(boxes) > 0:
            info = simple_image_path + ' ' + str(len(boxes)) +  ' ' + ' '.join(boxes) + '\n'
            outfile.write(info)

Invalid box: ('93', '334', '66', '68') at image positive\y283.jpg
Invalid box: ('91', '334', '63', '68') at image positive\y475.jpg
Invalid box: ('87', '-7', '70', '76') at image positive\y689.jpg
